In [ ]:
!unzip /content/ozetler.zip

In [ ]:
!pip install sentence-transformers faiss-cpu numpy sqlalchemy

In [ ]:

import os
import numpy as np
import faiss
import pickle
import sqlite3
from sentence_transformers import SentenceTransformer
import re

SUMMARIES_FOLDER = '/content/summaries'
DB_PATH = '/content/movies_final.db'
INDEX_FILE = 'movie_summaries_chunked.index'
CHUNK_MAP_FILE = 'chunk_map.pkl' 
CHUNK_SIZE = 300 
CHUNK_OVERLAP = 50  

def chunk_text_fixed_size(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap):
        chunk = ' '.join(words[i:i + chunk_size])
        if len(chunk.strip()) > 10:
            chunks.append(chunk)

    return chunks

def chunk_text_semantic(text, max_chunk_size=400):
    sentences = re.split(r'[.!?…]+', text)
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence = sentence.strip()
        if not sentence:
            continue

        sentence_length = len(sentence.split())

        if current_length + sentence_length > max_chunk_size and current_chunk:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_length
        else:
            current_chunk.append(sentence)
            current_length += sentence_length

    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')

    return chunks

print("Embedding modeli (emrecan/bert-base-turkish...) yükleniyor...")
model = SentenceTransformer('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
print("Model başarıyla yüklendi.")

try:
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT title FROM movies")
    db_titles = [row[0] for row in cursor.fetchall()]
    conn.close()
    print(f"SQL veritabanından {len(db_titles)} adet film başlığı okundu.")
except Exception as e:
    print(f"HATA: Veritabanı okunurken bir sorun oluştu: {e}")
    db_titles = []

all_embeddings = []
chunk_metadata = [] 

if db_titles:
    print(f"'{SUMMARIES_FOLDER}' klasöründeki özetler chunking ile işleniyor...")

    total_chunks = 0
    processed_movies = 0

    for title in db_titles:
        safe_filename = re.sub(r'[\\/*?:"<>|]', "_", title)
        file_path = os.path.join(SUMMARIES_FOLDER, f"{safe_filename}.txt")

        if os.path.exists(file_path):
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    summary_text = f.read().strip()

                if summary_text:
                    chunks = chunk_text_semantic(summary_text, max_chunk_size=CHUNK_SIZE)

                    for chunk_idx, chunk in enumerate(chunks):
                        if len(chunk.strip()) > 20:
                            try:
                                embedding = model.encode(chunk, convert_to_tensor=False)
                                all_embeddings.append(embedding)

                                chunk_metadata.append({
                                    'title': title,
                                    'chunk_index': chunk_idx,
                                    'chunk_text': chunk,
                                    'total_chunks': len(chunks)
                                })
                                total_chunks += 1
                            except Exception as e:
                                print(f"Chunk embedding hatası - {title} (chunk {chunk_idx}): {e}")

                    processed_movies += 1
                    if processed_movies % 10 == 0:
                        print(f"İşlenen film sayısı: {processed_movies}/{len(db_titles)}, Toplam chunk: {total_chunks}")

                else:
                    print(f"Uyarı: {safe_filename}.txt dosyası boş, atlanıyor.")

            except Exception as e:
                print(f"Hata: {title}.txt dosyası işlenirken sorun oluştu: {e}")
        else:
            print(f"Uyarı: '{safe_filename}.txt' için özet dosyası bulunamadı, atlanıyor.")

    if all_embeddings:
        embeddings_np = np.array(all_embeddings, dtype='float32')
        print(f"\n📊 Özet İstatistikler:")
        print(f"   • İşlenen film sayısı: {processed_movies}")
        print(f"   • Toplam chunk sayısı: {len(chunk_metadata)}")
        print(f"   • Ortalama chunk/film: {len(chunk_metadata)/processed_movies:.1f}")
        print(f"   • Embedding boyutu: {embeddings_np.shape[1]}")

        vector_dimension = embeddings_np.shape[1]
        index = faiss.IndexFlatL2(vector_dimension)

        print(f"\nFAISS indeksi {vector_dimension} boyutlu olarak oluşturuldu.")

        index.add(embeddings_np)
        print(f"{index.ntotal} chunk vektörü FAISS indeksine eklendi.")

        faiss.write_index(index, INDEX_FILE)
        print(f"FAISS indeksi '{INDEX_FILE}' dosyasına kaydedildi.")

        with open(CHUNK_MAP_FILE, 'wb') as f:
            pickle.dump(chunk_metadata, f)
        print(f"Chunk metadata '{CHUNK_MAP_FILE}' dosyasına kaydedildi.")

        print("\n✅ Chunking tabanlı vektör veritabanı oluşturma işlemi başarıyla tamamlandı!")

        if chunk_metadata:
            print(f"\n📝 İlk chunk örneği ({chunk_metadata[0]['title']}):")
            print(f"   Chunk {chunk_metadata[0]['chunk_index']+1}/{chunk_metadata[0]['total_chunks']}")
            print(f"   İçerik (ilk 200 karakter): {chunk_metadata[0]['chunk_text'][:200]}...")

    else:
        print("İşlenecek hiç geçerli chunk bulunamadı.")
else:
    print("Veritabanından film başlığı okunamadığı için işlem durduruldu.")

def search_similar_chunks(query, top_k=5):
    try:
        index = faiss.read_index(INDEX_FILE)
        with open(CHUNK_MAP_FILE, 'rb') as f:
            metadata = pickle.load(f)

        query_embedding = model.encode(query, convert_to_tensor=False)
        query_embedding = np.array([query_embedding], dtype='float32')

        distances, indices = index.search(query_embedding, top_k)

        results = []
        for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
            if idx < len(metadata):
                chunk_info = metadata[idx]
                results.append({
                    'rank': i + 1,
                    'similarity_score': float(1 / (1 + dist)), 
                    'movie_title': chunk_info['title'],
                    'chunk_index': chunk_info['chunk_index'],
                    'total_chunks': chunk_info['total_chunks'],
                    'chunk_text': chunk_info['chunk_text'],
                    'distance': float(dist)
                })

        return results

    except Exception as e:
        print(f"Arama hatası: {e}")
        return []

print(f"\n🔍 Test sorgusu çalıştırılıyor...")
test_query = "romantik aşk hikayesi"
results = search_similar_chunks(test_query, top_k=3)

if results:
    print(f"\n'{test_query}' sorgusu için en benzer 3 chunk:")
    for result in results:
        print(f"\n{result['rank']}. {result['movie_title']}")
        print(f"   Chunk {result['chunk_index']+1}/{result['total_chunks']} (Benzerlik: {result['similarity_score']:.3f})")
        print(f"   İçerik: {result['chunk_text'][:200]}...")

In [ ]:

import faiss
import numpy as np
import pickle

def optimize_index_for_cosine():
    index_l2 = faiss.read_index('movie_summaries_chunked.index')
    with open('chunk_map.pkl', 'rb') as f:
        metadata = pickle.load(f)

    print(f"Mevcut indeks: {index_l2.ntotal} vektör, {index_l2.d} boyut")

    vectors = np.zeros((index_l2.ntotal, index_l2.d), dtype='float32')
    for i in range(index_l2.ntotal):
        vectors[i] = index_l2.reconstruct(i)

    faiss.normalize_L2(vectors)

    index_cosine = faiss.IndexFlatIP(index_l2.d) 
    index_cosine.add(vectors)

    # Kaydet
    faiss.write_index(index_cosine, 'movie_summaries_cosine.index')
    print(f"✅ Cosine similarity indeksi kaydedildi: {index_cosine.ntotal} vektör")

    return index_cosine

def cosine_search(query, top_k=10, min_similarity=0.5):
    try:
        index = faiss.read_index('movie_summaries_cosine.index')
        with open('chunk_map.pkl', 'rb') as f:
            metadata = pickle.load(f)

        query_vector = model.encode(query, convert_to_tensor=False)
        query_vector = np.array([query_vector], dtype='float32')
        faiss.normalize_L2(query_vector)

        similarities, indices = index.search(query_vector, min(top_k * 2, len(metadata)))

        results = []
        for i, (sim, idx) in enumerate(zip(similarities[0], indices[0])):
            if sim >= min_similarity and idx < len(metadata):
                chunk_info = metadata[idx]
                results.append({
                    'rank': len(results) + 1,
                    'similarity_score': float(sim),
                    'movie_title': chunk_info['title'],
                    'chunk_index': chunk_info['chunk_index'],
                    'total_chunks': chunk_info['total_chunks'],
                    'chunk_text': chunk_info['chunk_text'],
                })

        return results[:top_k]

    except FileNotFoundError:
        print("Cosine indeksi bulunamadı. optimize_index_for_cosine() fonksiyonunu çalıştırın.")
        return []
    except Exception as e:
        print(f"Arama hatası: {e}")
        return []

print("🔧 İndeks cosine similarity için optimize ediliyor...")
optimized_index = optimize_index_for_cosine()

print("\n🔍 Cosine similarity ile test...")
test_queries = [
    "pandemide evinde kalıp şarkılar çalan bi adam vardı komedyen",
]

for query in test_queries:
    print(f"\n📝 Sorgu: '{query}'")
    results = cosine_search(query, top_k=5, min_similarity=0.3)

    if results:
        for result in results:
            print(f"   {result['rank']}. {result['movie_title']} (Skor: {result['similarity_score']:.3f})")
            print(f"      Chunk {result['chunk_index']+1}: {result['chunk_text'][:100]}...")
    else:
        print("   ❌ Uygun sonuç bulunamadı (min_similarity=0.3)")

print("\n✅ Optimizasyon tamamlandı!")